In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
var date = DateTime.Now.ToString("ddMMyyyy");
date

24082023

open a database

In [3]:
string dbPath = @"WedgeFlow_db_" + date ;
var database = OpenOrCreateDatabase(dbPath);

Opening existing database 'WedgeFlow_db_24082023'.


run WedgeFlow simulation and safe to database

In [4]:
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
    dbPath:dbPath,
    optiLSDegree: 1,
    lsDegree: 1,
    shocksetup: ApplicationWithIDT.GetLevelSet.FromParams,
    optiLevelSetType: OptiLevelSetType.GlobalLevelSet,
    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
    MaxIterations: 200,
    dgDegree: 0,
    numOfCellsX: 15,
    numOfCellsY: 10,
    initialAngle_shockLS: 32,
    PlotInterval: -1,
    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
    agg: 0.4
);
p.Init(C);
p.RunSolverMode();


rm;
Grid Edge Tags changed.
Session ID: bef6f26f-ce1d-4af5-abd5-b2f47b6a80db, DB path: 'WedgeFlow_db_24082023'
Session directory 'WedgeFlow_db_24082023\sessions\bef6f26f-ce1d-4af5-abd5-b2f47b6a80db'.
Initial Value : l2: |r|= 1.0778378E+01, |f_err|=2.4570273E+01, | f_phi |=0E+00,

Starting Opt-Iter No.1: 
alpha, M, predM  1, 2317.9523861937532, 14474.78795105228
l2: ||R0||= 3.0354125E+00, f_err=1.120319E+01, f_phi=0E+00, Alpha = 1E+00, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.92E+02, QuadOrder=2
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.2: 
alpha, M, predM  1, 462.9455842382383, 1765.9344254969371
l2: ||R0||= 1.0703392E+00, f_err=8.219137E+00, f_phi=0E+00, Alpha = 1E+00, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.25E+02, QuadOrder=2
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.3: 
alpha, ||R1||  1, Error 
alpha, ||R1||  0.5, Error 
alpha, M, predM  0.25, 612.8418096532504, 342.4587559787904
alpha, M, predM 

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [5]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);

## Enthalpy error

enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

As we have uniform inflow the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [10]:
p.GetResEnthalpyPlot(si,EEN).dataGroups[2].Values.SaveToTextFile("siWF2EnthalpyErrors.txt");
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M988.5,57.1 L1041.9,57.1 M53.9,53.8 L81.3,63.7 L108.7,67.6 L136.1,67.7 L163.5,68.2 L190.9,68.2
 L218.3,69.1 L245.7,69.1 L273.1,69.4 L300.5,69.5 L328.0,70.0 L355.4,70.8 L382.8,71.6 L410.2,72.1
 L437.6,72.5 L465.0,73.0 L492.4,75.3 L519.8,75.9 L547.2,76.6 L574.6,77.0 L602.0,79.9 L629.4,80.6
 L656.8,80.8 L684.2,83.8 L711.6,87.2 L739.0,88.8 L766.4,112.4 L793.8,134.4 L821.2,232.0 L848.6,330.2
 L876.1,330.4 L903.5,330.0 L930.9,330.0 L958.3,330.0 L985.7,330.0 L1013.1,330.0 L1040.5,330.0 L1067.9,330.0
 L1095.3,330.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M988.5,81.1 L1041.9,81.1 M53.9,64.2 L81.3,80.1 L108.7,93.3 L136.1,92.9 L163.5,94.3 L190.9,94.4
 L218.3,98.5 L245.7,98.1 L273.1,99.1 L300.5,98.7 L328.0,125.6 L355.4,127.8 L382.8,115.5 L410.2,104.2
 L437.6,104.9 L465.0,105.3 L492.4,108.6 L519.8,110.2 L547.2,101.5 L574.6,101.9 L602.0,105.7 L629.4,106.4
 L656.8,106.6 L684.2,104.5 L711.6,107.8 L739.0,109.5 L766.4,120.3 L793.8,146.0 L821.2,249.1 L848.6,352.4
 L876.1,364.7 L903.5,380.1 L930.9,379.8 L958.3,379.8 L985.7,379.8 L1013.1,379.8 L1040.5,379.8 L1067.9,379.8
 L1095.3,379.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||herr|| 
 
 
 ||h err || 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M988.5,105.1 L1041.9,105.1 M53.9,193.8 L81.3,167.3 L108.7,167.4 L136.1,168.2 L163.5,168.4 L190.9,168.4
 L218.3,170.3 L245.7,172.7 L273.1,173.0 L300.5,173.2 L328.0,167.9 L355.4,169.2 L382.8,170.4 L410.2,171.6
 L437.6,172.3 L465.0,172.2 L492.4,175.1 L519.8,175.8 L547.2,176.9 L574.6,177.3 L602.0,180.3 L629.4,181.0
 L656.8,181.2 L684.2,186.0 L711.6,189.5 L739.0,191.1 L766.4,211.8 L793.8,218.7 L821.2,317.1 L848.6,419.0
 L876.1,417.3 L903.5,417.1 L930.9,417.0 L958.3,417.0 L985.7,417.0 L1013.1,417.0 L1040.5,417.0 L1067.9,417.0
 L1095.3,417.0 '/>

In [ ]:
p.PlotShadowFields(si);